In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
df = pd.read_csv('../data/housing.csv', delim_whitespace=True, header=None) # delim_whitespace 있어야 제대로 읽을 수 있음
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       506 non-null    float64
 1   1       506 non-null    float64
 2   2       506 non-null    float64
 3   3       506 non-null    int64  
 4   4       506 non-null    float64
 5   5       506 non-null    float64
 6   6       506 non-null    float64
 7   7       506 non-null    float64
 8   8       506 non-null    int64  
 9   9       506 non-null    float64
 10  10      506 non-null    float64
 11  11      506 non-null    float64
 12  12      506 non-null    float64
 13  13      506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [4]:
df.corr()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
1,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445
2,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725
3,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260
4,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
5,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,0.695360
6,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
7,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
8,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626
9,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536


In [9]:
dataset = df.values
X = dataset[:,:13]
Y = dataset[:,13]

print(X.shape, Y.shape)

(506, 13) (506,)


In [8]:
seed=1234
np.random.seed(seed)
tf.random.set_seed(seed)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(404, 13) (102, 13) (404,) (102,)


In [11]:
model = Sequential()
model.add(Dense(30, input_dim=13, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

In [12]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['MAE']) # Mean Absolute Error

In [14]:
model.fit(X_train, Y_train, epochs=500, batch_size=10)

Epoch 1/500
41/41 [==============================] - 0s 2ms/step - loss: 17.6544 - MAE: 3.0588
Epoch 2/500
41/41 [==============================] - 0s 2ms/step - loss: 19.6476 - MAE: 3.1613
Epoch 3/500
41/41 [==============================] - 0s 2ms/step - loss: 17.3261 - MAE: 3.0607
Epoch 4/500
41/41 [==============================] - 0s 2ms/step - loss: 16.7288 - MAE: 2.9556
Epoch 5/500
41/41 [==============================] - 0s 2ms/step - loss: 16.7443 - MAE: 2.9411
Epoch 6/500
41/41 [==============================] - 0s 2ms/step - loss: 17.9983 - MAE: 3.0471
Epoch 7/500
41/41 [==============================] - 0s 2ms/step - loss: 15.3895 - MAE: 2.8504
Epoch 8/500
41/41 [==============================] - 0s 2ms/step - loss: 14.8571 - MAE: 2.7613
Epoch 9/500
41/41 [==============================] - 0s 2ms/step - loss: 15.9563 - MAE: 2.9839: 0s - loss: 14.6484 - MAE: 2.85
Epoch 10/500
41/41 [==============================] - 0s 2ms/step - loss: 15.2677 - MAE: 2.8652
Epoch 11/500
41/4

41/41 [==============================] - 0s 2ms/step - loss: 13.7685 - MAE: 2.6799
Epoch 86/500
41/41 [==============================] - 0s 2ms/step - loss: 15.8118 - MAE: 2.9073
Epoch 87/500
41/41 [==============================] - 0s 2ms/step - loss: 15.2948 - MAE: 2.8023
Epoch 88/500
41/41 [==============================] - 0s 2ms/step - loss: 12.9445 - MAE: 2.6091
Epoch 89/500
41/41 [==============================] - 0s 2ms/step - loss: 15.6004 - MAE: 2.7980
Epoch 90/500
41/41 [==============================] - 0s 2ms/step - loss: 16.9578 - MAE: 2.9722
Epoch 91/500
41/41 [==============================] - 0s 2ms/step - loss: 13.4804 - MAE: 2.6757
Epoch 92/500
41/41 [==============================] - 0s 3ms/step - loss: 14.0381 - MAE: 2.7241
Epoch 93/500
41/41 [==============================] - 0s 2ms/step - loss: 13.8152 - MAE: 2.6519
Epoch 94/500
41/41 [==============================] - 0s 2ms/step - loss: 13.9908 - MAE: 2.5771
Epoch 95/500
41/41 [==============================] -

41/41 [==============================] - ETA: 0s - loss: 12.3366 - MAE: 2.37 - 0s 2ms/step - loss: 11.5156 - MAE: 2.3683
Epoch 170/500
41/41 [==============================] - 0s 2ms/step - loss: 13.2280 - MAE: 2.5912
Epoch 171/500
41/41 [==============================] - 0s 2ms/step - loss: 12.5853 - MAE: 2.5452
Epoch 172/500
41/41 [==============================] - 0s 2ms/step - loss: 12.1980 - MAE: 2.4931
Epoch 173/500
41/41 [==============================] - 0s 2ms/step - loss: 12.0820 - MAE: 2.5300
Epoch 174/500
41/41 [==============================] - 0s 2ms/step - loss: 11.2564 - MAE: 2.4480
Epoch 175/500
41/41 [==============================] - 0s 2ms/step - loss: 11.0970 - MAE: 2.3333
Epoch 176/500
41/41 [==============================] - 0s 2ms/step - loss: 10.5258 - MAE: 2.3446
Epoch 177/500
41/41 [==============================] - 0s 2ms/step - loss: 12.1686 - MAE: 2.4457
Epoch 178/500
41/41 [==============================] - 0s 2ms/step - loss: 11.3232 - MAE: 2.3912
Epoch 

41/41 [==============================] - 0s 2ms/step - loss: 10.9842 - MAE: 2.4079
Epoch 254/500
41/41 [==============================] - 0s 2ms/step - loss: 10.2781 - MAE: 2.2725
Epoch 255/500
41/41 [==============================] - 0s 2ms/step - loss: 11.0843 - MAE: 2.4047
Epoch 256/500
41/41 [==============================] - 0s 2ms/step - loss: 11.0225 - MAE: 2.4202
Epoch 257/500
41/41 [==============================] - 0s 2ms/step - loss: 11.3947 - MAE: 2.4027
Epoch 258/500
41/41 [==============================] - 0s 2ms/step - loss: 10.5426 - MAE: 2.3474
Epoch 259/500
41/41 [==============================] - 0s 2ms/step - loss: 12.8432 - MAE: 2.6450
Epoch 260/500
41/41 [==============================] - 0s 2ms/step - loss: 11.5281 - MAE: 2.4313
Epoch 261/500
41/41 [==============================] - 0s 2ms/step - loss: 9.8427 - MAE: 2.2740
Epoch 262/500
41/41 [==============================] - 0s 2ms/step - loss: 10.1970 - MAE: 2.2607
Epoch 263/500
41/41 [========================

41/41 [==============================] - 0s 2ms/step - loss: 10.8239 - MAE: 2.3367
Epoch 338/500
41/41 [==============================] - 0s 2ms/step - loss: 9.2446 - MAE: 2.1884
Epoch 339/500
41/41 [==============================] - 0s 2ms/step - loss: 10.4587 - MAE: 2.3056
Epoch 340/500
41/41 [==============================] - 0s 2ms/step - loss: 8.8807 - MAE: 2.1663
Epoch 341/500
41/41 [==============================] - 0s 2ms/step - loss: 11.6326 - MAE: 2.4815
Epoch 342/500
41/41 [==============================] - 0s 2ms/step - loss: 10.5955 - MAE: 2.3491
Epoch 343/500
41/41 [==============================] - 0s 2ms/step - loss: 10.3875 - MAE: 2.2977
Epoch 344/500
41/41 [==============================] - 0s 2ms/step - loss: 10.0289 - MAE: 2.2192
Epoch 345/500
41/41 [==============================] - 0s 2ms/step - loss: 8.9802 - MAE: 2.1565
Epoch 346/500
41/41 [==============================] - 0s 2ms/step - loss: 8.7500 - MAE: 2.1416
Epoch 347/500
41/41 [===========================

41/41 [==============================] - 0s 2ms/step - loss: 9.1942 - MAE: 2.1538
Epoch 421/500
41/41 [==============================] - 0s 2ms/step - loss: 8.9518 - MAE: 2.1659
Epoch 422/500
41/41 [==============================] - 0s 2ms/step - loss: 8.4146 - MAE: 2.1387
Epoch 423/500
41/41 [==============================] - 0s 2ms/step - loss: 8.6151 - MAE: 2.0912
Epoch 424/500
41/41 [==============================] - 0s 2ms/step - loss: 8.6367 - MAE: 2.0857
Epoch 425/500
41/41 [==============================] - 0s 2ms/step - loss: 8.8761 - MAE: 2.1734
Epoch 426/500
41/41 [==============================] - 0s 3ms/step - loss: 8.1943 - MAE: 2.0676
Epoch 427/500
41/41 [==============================] - 0s 3ms/step - loss: 9.5337 - MAE: 2.1873
Epoch 428/500
41/41 [==============================] - 0s 2ms/step - loss: 8.6345 - MAE: 2.1402
Epoch 429/500
41/41 [==============================] - 0s 2ms/step - loss: 8.9539 - MAE: 2.2187
Epoch 430/500
41/41 [==============================] -

In [16]:
result1 = model.evaluate(X_train, Y_train, verbose=0)
result2 = model.evaluate(X_test, Y_test, verbose=0)

print('학습평가:', result1)
print('검증평가:', result2)

학습평가: [6.92995023727417, 1.8963435888290405]
검증평가: [11.875271797180176, 2.5277085304260254]


In [ ]:
# 전체 코드
df = pd.read_csv('../data/housing.csv', delim_whitespace=True, header=None)

dataset = df.values
X = dataset[:,:13]
Y = dataset[:,13]

seed=1234
np.random.seed(seed)
tf.random.set_seed(seed)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)

model = Sequential()
model.add(Dense(30, input_dim=13, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['MAE']) # Mean Absolute Error

model.fit(X_train, Y_train, epochs=500, batch_size=10)

In [24]:
result1 = model.evaluate(X_train, Y_train, verbose=0)
result2 = model.evaluate(X_test, Y_test, verbose=0)

print('학습평가:', result1)
print('검증평가:', result2)

학습평가: [6.92995023727417, 1.8963435888290405]
검증평가: [11.875271797180176, 2.5277085304260254]


In [ ]:
# 미래 결과 예측

In [21]:
print(X_test.shape)

Y_pred = model.predict(X_test)
print(Y_pred.shape)

Y_pred = Y_pred.flatten()
print(Y_pred.shape) # 모델의 예측값
print(Y_test.shape) # 실제 정답

(102, 13)
(102, 1)
(102,)
(102,)


In [23]:
for val1, val2 in zip(Y_test, Y_pred):
    print('실제 집값: %5.2f, 예측 집값: %5.2f, 오차: %5.2f' % (val1, val2, val1-val2))

실제 집값: 33.00, 예측 집값: 22.74, 오차: 10.26
실제 집값: 27.50, 예측 집값: 21.58, 오차:  5.92
실제 집값:  5.60, 예측 집값: 11.14, 오차: -5.54
실제 집값: 21.20, 예측 집값: 23.50, 오차: -2.30
실제 집값: 14.90, 예측 집값: 14.13, 오차:  0.77
실제 집값: 22.30, 예측 집값: 23.78, 오차: -1.48
실제 집값: 18.80, 예측 집값: 18.43, 오차:  0.37
실제 집값: 14.60, 예측 집값: 14.55, 오차:  0.05
실제 집값: 19.40, 예측 집값: 23.76, 오차: -4.36
실제 집값: 32.00, 예측 집값: 30.90, 오차:  1.10
실제 집값: 13.80, 예측 집값: 16.11, 오차: -2.31
실제 집값: 21.70, 예측 집값: 22.93, 오차: -1.23
실제 집값: 22.60, 예측 집값: 24.09, 오차: -1.49
실제 집값: 18.40, 예측 집값: 17.69, 오차:  0.71
실제 집값: 20.50, 예측 집값: 17.71, 오차:  2.79
실제 집값: 22.20, 예측 집값: 22.49, 오차: -0.29
실제 집값: 10.80, 예측 집값: 11.91, 오차: -1.11
실제 집값: 22.50, 예측 집값: 18.30, 오차:  4.20
실제 집값: 13.80, 예측 집값: 16.33, 오차: -2.53
실제 집값:  5.00, 예측 집값:  9.87, 오차: -4.87
실제 집값: 32.90, 예측 집값: 32.85, 오차:  0.05
실제 집값: 18.60, 예측 집값: 21.05, 오차: -2.45
실제 집값: 16.80, 예측 집값: 20.85, 오차: -4.05
실제 집값: 27.10, 예측 집값: 17.70, 오차:  9.40
실제 집값: 22.90, 예측 집값: 23.05, 오차: -0.15
실제 집값: 19.60, 예측 집값: 22.18, 오차: -2.58
실제 집값: 22.70

In [25]:
df = pd.read_csv('../data/house_train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [26]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
'''
    # 결측치 칼럼 #
    3   LotFrontage    1201 non-null   float64
    6   Alley          91 non-null     object 
    25  MasVnrType     1452 non-null   object 
    26  MasVnrArea     1452 non-null   float64
    30  BsmtQual       1423 non-null   object 
    31  BsmtCond       1423 non-null   object 
    32  BsmtExposure   1422 non-null   object 
    33  BsmtFinType1   1423 non-null   object 
    35  BsmtFinType2   1422 non-null   object 
    42  Electrical     1459 non-null   object 
    57  FireplaceQu    770 non-null    object 
    58  GarageType     1379 non-null   object 
    59  GarageYrBlt    1379 non-null   float64
    60  GarageFinish   1379 non-null   object 
    63  GarageQual     1379 non-null   object 
    64  GarageCond     1379 non-null   object 
    72  PoolQC         7 non-null      object 
    73  Fence          281 non-null    object 
    74  MiscFeature    54 non-null     object 
 
    1. 데이터 셋에는 문자열이 포함될 수 없다.
        1) 문자열 칼럼에 있는 모든 결측치를 'N'으로 변환한다.
        2) 모든 문자열 칼럼을 레이블 인코딩 작업을 수행하여 숫자 칼럼으로 변환한다.
    
    2. 숫자 칼럼의 결측치를 어떤 값(평균 값)으로 채우기
        1) 
'''

In [36]:
# 데이터프레임에서 모든 문자열 칼럼만 선별하는 작업
str_columns = []

for col_name in df.columns:
    if df[col_name].dtype == 'object':
        str_columns.append(col_name)
        
# 문자열 칼럼에 모든 결측치를 'N'으로 채우는 작업
for col_name in str_columns:
    df[col_name].fillna('N', inplace=True) # inplace=True 해야 원본 데이터 유지

# 모든 문자열 칼럼을 레이블 인코딩 작업을 수행하여 숫자 칼럼으로 변환하는 작업
for col_name in str_columns:
    e = LabelEncoder()
    e.fit(df[col_name])
    df[col_name] = e.transform(df[col_name])
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int32  
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int32  
 6   Alley          1460 non-null   int32  
 7   LotShape       1460 non-null   int32  
 8   LandContour    1460 non-null   int32  
 9   Utilities      1460 non-null   int32  
 10  LotConfig      1460 non-null   int32  
 11  LandSlope      1460 non-null   int32  
 12  Neighborhood   1460 non-null   int32  
 13  Condition1     1460 non-null   int32  
 14  Condition2     1460 non-null   int32  
 15  BldgType       1460 non-null   int32  
 16  HouseStyle     1460 non-null   int32  
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
'''
문제] 숫자 칼럼의 결측치를 평균 값으로 채우는 코드를 작성하세요.
'''

In [39]:
print(df['LotFrontage'].isnull().sum()) # 결측치 259개
print(df['Id'].isnull().sum()) # 결측치 0개

259
0


In [41]:
for col_name in df.columns:
    if df[col_name].isnull().sum() != 0:
        df[col_name].fillna(df[col_name].mean(), inplace=True)
        
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int32  
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int32  
 6   Alley          1460 non-null   int32  
 7   LotShape       1460 non-null   int32  
 8   LandContour    1460 non-null   int32  
 9   Utilities      1460 non-null   int32  
 10  LotConfig      1460 non-null   int32  
 11  LandSlope      1460 non-null   int32  
 12  Neighborhood   1460 non-null   int32  
 13  Condition1     1460 non-null   int32  
 14  Condition2     1460 non-null   int32  
 15  BldgType       1460 non-null   int32  
 16  HouseStyle     1460 non-null   int32  
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
'''
문제2] 칼럼 간의 상관도를 조사하여 'SalePrice' 값의 상관도가 20%(+,-) 이상인 칼럼들만 포함하는
        데이터프레임을 새롭게 만드는 코드를 작성하세요.
'''

In [51]:
df_corr = df.corr()
list1 = list(df_corr[df_corr['SalePrice'] >= 0.2].index)
list2 = list(df_corr[df_corr['SalePrice'] <= -0.2].index)
col_list = list1 + list2

df = df[col_list]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotFrontage    1460 non-null   float64
 1   LotArea        1460 non-null   int64  
 2   Neighborhood   1460 non-null   int32  
 3   OverallQual    1460 non-null   int64  
 4   YearBuilt      1460 non-null   int64  
 5   YearRemodAdd   1460 non-null   int64  
 6   RoofStyle      1460 non-null   int32  
 7   MasVnrArea     1460 non-null   float64
 8   Foundation     1460 non-null   int32  
 9   BsmtFinSF1     1460 non-null   int64  
 10  BsmtUnfSF      1460 non-null   int64  
 11  TotalBsmtSF    1460 non-null   int64  
 12  CentralAir     1460 non-null   int32  
 13  Electrical     1460 non-null   int32  
 14  1stFlrSF       1460 non-null   int64  
 15  2ndFlrSF       1460 non-null   int64  
 16  GrLivArea      1460 non-null   int64  
 17  BsmtFullBath   1460 non-null   int64  
 18  FullBath

In [53]:
dataset = df.values
X = dataset[:,:39]
Y = dataset[:,39]

seed=1234
np.random.seed(seed)
tf.random.set_seed(seed)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)

model = Sequential()
model.add(Dense(30, input_dim=39, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['MAE']) # Mean Absolute Error

model.fit(X_train, Y_train, epochs=500, batch_size=10)

Epoch 1/500
117/117 [==============================] - 1s 2ms/step - loss: 893877.3750 - MAE: 327.8831
Epoch 2/500
117/117 [==============================] - 0s 2ms/step - loss: 1274.8545 - MAE: 26.7769
Epoch 3/500
117/117 [==============================] - 0s 2ms/step - loss: 570.7368 - MAE: 16.8046
Epoch 4/500
117/117 [==============================] - 0s 2ms/step - loss: 406.5643 - MAE: 14.6023
Epoch 5/500
117/117 [==============================] - 0s 2ms/step - loss: 190.9206 - MAE: 10.0524
Epoch 6/500
117/117 [==============================] - 0s 2ms/step - loss: 141.8262 - MAE: 8.7315
Epoch 7/500
117/117 [==============================] - 0s 2ms/step - loss: 85.5732 - MAE: 6.6660
Epoch 8/500
117/117 [==============================] - 0s 2ms/step - loss: 72.0117 - MAE: 6.1805
Epoch 9/500
117/117 [==============================] - 0s 2ms/step - loss: 84.8093 - MAE: 6.6665
Epoch 10/500
117/117 [==============================] - 0s 2ms/step - loss: 78.4921 - MAE: 6.4717
Epoch 11/500


117/117 [==============================] - 0s 2ms/step - loss: 42.3460 - MAE: 4.3757
Epoch 85/500
117/117 [==============================] - 0s 2ms/step - loss: 34.4936 - MAE: 3.5034
Epoch 86/500
117/117 [==============================] - 0s 3ms/step - loss: 23.8166 - MAE: 3.3476: 0s - loss: 28.2360 - M
Epoch 87/500
117/117 [==============================] - 0s 3ms/step - loss: 192.3268 - MAE: 9.4889
Epoch 88/500
117/117 [==============================] - 0s 2ms/step - loss: 14.6653 - MAE: 2.6771
Epoch 89/500
117/117 [==============================] - 0s 2ms/step - loss: 6.7854 - MAE: 1.9812
Epoch 90/500
117/117 [==============================] - 0s 2ms/step - loss: 5.1867 - MAE: 1.7178
Epoch 91/500
117/117 [==============================] - 0s 2ms/step - loss: 9.5210 - MAE: 2.1771
Epoch 92/500
117/117 [==============================] - 0s 3ms/step - loss: 9.5581 - MAE: 2.3518
Epoch 93/500
117/117 [==============================] - 0s 2ms/step - loss: 6.4418 - MAE: 1.9014
Epoch 94/500


117/117 [==============================] - 0s 2ms/step - loss: 2.7660 - MAE: 1.3103
Epoch 167/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4481 - MAE: 0.9846
Epoch 168/500
117/117 [==============================] - 0s 2ms/step - loss: 1.8702 - MAE: 1.0942
Epoch 169/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4359 - MAE: 0.9795
Epoch 170/500
117/117 [==============================] - 0s 2ms/step - loss: 1.8417 - MAE: 1.1036
Epoch 171/500
117/117 [==============================] - 0s 2ms/step - loss: 1.5801 - MAE: 1.0227
Epoch 172/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4717 - MAE: 0.9957
Epoch 173/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4576 - MAE: 0.9907
Epoch 174/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4439 - MAE: 0.9846
Epoch 175/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4038 - MAE: 0.9752
Epoch 176/500
117/117 [===========

117/117 [==============================] - 0s 2ms/step - loss: 1.3998 - MAE: 0.9821
Epoch 250/500
117/117 [==============================] - 0s 2ms/step - loss: 1.3999 - MAE: 0.9821
Epoch 251/500
117/117 [==============================] - 0s 2ms/step - loss: 1.3998 - MAE: 0.9853
Epoch 252/500
117/117 [==============================] - 0s 3ms/step - loss: 1.3999 - MAE: 0.9816
Epoch 253/500
117/117 [==============================] - 0s 2ms/step - loss: 1.3996 - MAE: 0.9854
Epoch 254/500
117/117 [==============================] - 0s 2ms/step - loss: 1.3998 - MAE: 0.9853
Epoch 255/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4000 - MAE: 0.9855
Epoch 256/500
117/117 [==============================] - 0s 2ms/step - loss: 1.3997 - MAE: 0.9834
Epoch 257/500
117/117 [==============================] - 0s 3ms/step - loss: 1.3997 - MAE: 0.9831
Epoch 258/500
117/117 [==============================] - 0s 2ms/step - loss: 1.3998 - MAE: 0.9855
Epoch 259/500
117/117 [===========

117/117 [==============================] - 0s 2ms/step - loss: 1.4000 - MAE: 0.9838
Epoch 331/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4009 - MAE: 0.9884
Epoch 332/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4003 - MAE: 0.9778
Epoch 333/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4000 - MAE: 0.9801
Epoch 334/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4002 - MAE: 0.9865
Epoch 335/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4006 - MAE: 0.9883
Epoch 336/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4012 - MAE: 0.9863
Epoch 337/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4003 - MAE: 0.9749
Epoch 338/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4003 - MAE: 0.9887
Epoch 339/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4007 - MAE: 0.9887
Epoch 340/500
117/117 [===========

117/117 [==============================] - 0s 2ms/step - loss: 1.4000 - MAE: 0.9872
Epoch 414/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4006 - MAE: 0.9779
Epoch 415/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4013 - MAE: 0.9829
Epoch 416/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4023 - MAE: 0.9808
Epoch 417/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4004 - MAE: 0.9889
Epoch 418/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4010 - MAE: 0.9834
Epoch 419/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4013 - MAE: 0.9872
Epoch 420/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4003 - MAE: 0.9800
Epoch 421/500
117/117 [==============================] - 0s 3ms/step - loss: 1.4011 - MAE: 0.9808
Epoch 422/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4003 - MAE: 0.9904
Epoch 423/500
117/117 [===========

117/117 [==============================] - 0s 3ms/step - loss: 1.4000 - MAE: 0.9860
Epoch 495/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4007 - MAE: 0.9810
Epoch 496/500
117/117 [==============================] - 0s 3ms/step - loss: 1.4000 - MAE: 0.9787
Epoch 497/500
117/117 [==============================] - 0s 2ms/step - loss: 1.4007 - MAE: 0.9892
Epoch 498/500
117/117 [==============================] - 0s 3ms/step - loss: 1.4002 - MAE: 0.9890
Epoch 499/500
117/117 [==============================] - 0s 3ms/step - loss: 1.4008 - MAE: 0.9827
Epoch 500/500
117/117 [==============================] - 0s 3ms/step - loss: 1.3999 - MAE: 0.9832
